In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
import gspread
from google.auth import default
#autenticating to google
auth.authenticate_user()

creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import os

# for dirname, _, filenames in os.walk('/content/drive/My Drive/data/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# worksheet = gc.open('train1').sheet1
# rows = worksheet.get_all_values()|
# df = pd.DataFrame(rows)
# df.columns = df.iloc[0]
# train_data = df.iloc[1:]

female_dataset = pd.read_csv("/content/drive/My Drive/Autocomplete/Autocomplete Dataset - BiasedFemale.csv")
male_dataset = pd.read_csv("/content/drive/My Drive/Autocomplete/Autocomplete Dataset - BiasedMale.csv")
female_dataset["Comments"] = female_dataset["Comments"].str.replace('\r\n', '')
male_dataset["Comments"] = male_dataset["Comments"].str.replace('\r\n', '')

In [ ]:
!pip install markovify
import pandas as pd
import markovify

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 7.1 MB/s eta 0:00:00
  Created wheel for markovify: filename=markovify-0.9.4-py3-none-any.whl size=18605 sha256=1b0cb02a337ba9483ddc8fb00f4c32f0395cef0ab3735a9092f19a8888294b7d
  Stored in directory: /root/.cache/pip/wheels/ca/8c/c5/41413e24c484f883a100c63ca7b3b0362b7c6f6eb6d7c9cc7f
Successfully built markovify


In [ ]:
text_model = markovify.NewlineText(female_dataset.Comments, state_size = 1)
text_model = markovify.NewlineText(male_dataset.Comments, state_size = 1)

In [ ]:
def calculate_token_frequencies(dataset):
    frequency_dict = {}

    for text in dataset.Comments.tolist():
        tokens = text.split()
        for token in tokens:
            if token in frequency_dict:
                frequency_dict[token] += 1
            else:
                frequency_dict[token] = 1

    return frequency_dict

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
def keep_adjectives(words):
    tagged_words = nltk.pos_tag(words)
    adjectives = [word for word, pos in tagged_words if pos.startswith('JJ')]
    return adjectives

In [ ]:
def calculate_token_frequencies(dataset):
    frequency_dict = {}

    for text in dataset.Comments.tolist():
        tokens = text.split()
        tokens = keep_adjectives(tokens)
        for token in tokens:
            if token in frequency_dict:
                frequency_dict[token] += 1
            else:
                frequency_dict[token] = 1

    return frequency_dict

In [ ]:
female_word_frequency = dict(sorted(calculate_token_frequencies(female_dataset).items(), key=lambda item: item[1], reverse=True))
female_word_frequency

{'inclined': 5,
 'emotional': 4,
 'focused': 3,
 'more': 3,
 'social': 3,
 'natural': 2,
 'sensitive': 2,
 'empathetic.': 2,
 'strong': 2,
 'relationship-oriented': 2,
 'capable': 2,
 'such': 2,
 'likely': 2,
 'submissive': 2,
 'passive,': 2,
 'harmonious': 2,
 'powerful': 1,
 'significant': 1,
 'assertive': 1,
 'confident': 1,
 'interested': 1,
 'technical': 1,
 'organizational': 1,
 'decision-making': 1,
 'resilient,': 1,
 'talkative': 1,
 'enjoy': 1,
 'detail-oriented': 1,
 'small': 1,
 'work-life': 1,
 'flexible': 1,
 'artistic': 1,
 'creative': 1,
 'pleasing': 1,
 'intelligent': 1,
 'various': 1,
 'rational': 1,
 'sensitive,': 1}

In [ ]:
stopwords = set(female_word_frequency.keys())
stopwords.update(["br", "href"])
textt = " ".join(review for review in df.Text)
wordcloud = WordCloud(stopwords=stopwords).generate(textt)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud11.png')
plt.show()

NameError: ignored

In [ ]:
male_word_frequency = dict(sorted(calculate_token_frequencies(male_dataset).items(), key=lambda item: item[1], reverse=True))
male_word_frequency

{'logical': 4,
 'interested': 4,
 'inclined': 4,
 'financial': 3,
 'rational': 3,
 'primary': 2,
 'capable': 2,
 'natural': 2,
 'physical': 2,
 'emotional': 2,
 'focused': 2,
 'nurturing': 2,
 'competitive': 2,
 'likely': 2,
 'great': 2,
 'more': 2,
 'independent': 2,
 'self-reliant.': 2,
 'stronger': 1,
 'manual': 1,
 'empathy,': 1,
 'assertive': 1,
 'confident': 1,
 'effective': 1,
 'analytical': 1,
 'valuable': 1,
 'well-rounded': 1,
 'domestic': 1,
 'potential': 1,
 'involved': 1,
 'active': 1,
 'adventurous': 1,
 'skilled': 1,
 'interpersonal': 1,
 'artistic': 1,
 'creative': 1}

### Autocomplete for Women

In [ ]:
import ipywidgets as widgets
from IPython.display import display,clear_output
from ipywidgets import interact
text_model = markovify.NewlineText(female_dataset.Comments, state_size = 1)

def autocomplete(search_term):
  autocomplete_options = []
  for i in range(10):
      sentence = text_model.make_sentence_with_start(beginning=search_term)
      if sentence:
        autocomplete_options.append(sentence)
  return autocomplete_options

input_str = input("Search: ")
autocomplete_options = autocomplete(input_str)
out = widgets.Output(layout={'border': '1px solid black'})
with out:
    for sentence in autocomplete_options:
        print(sentence)
display(out)

Search: Women


Output(layout=Layout(border='1px solid black'))

### Autocomplete for Men

In [ ]:
import ipywidgets as widgets
from IPython.display import display,clear_output
from ipywidgets import interact
text_model = markovify.NewlineText(male_dataset.Comments, state_size = 1)

def autocomplete(search_term):
  autocomplete_options = []
  for i in range(10):
      sentence = text_model.make_sentence_with_start(beginning=search_term)
      if sentence:
        autocomplete_options.append(sentence)
  return autocomplete_options

input_str = input("Search: ")
autocomplete_options = autocomplete(input_str)
out = widgets.Output(layout={'border': '1px solid black'})
with out:
    for sentence in autocomplete_options:
        print(sentence)
display(out)

Search: Men


Output(layout=Layout(border='1px solid black'))